In [127]:
import pickle
import numpy as np
from rul_calce_data import individual_battery_run 
from utility import cal_ttf

def rul_battery(t, y, name):
    # Calculate true TTF.
    threshold = .7*1.1
    true_ttf, idx_ttf = cal_ttf(t, y, threshold)
    t = t[:idx_ttf+10]
    y = y[:idx_ttf+10]
    T = len(t) # Number of time steps
    
    # Define the Parameters.
    # For the PF.
    sigma_u = np.array([1e-2, 1e-5, 1e-4, 1e-3])
    sigma_v = 1e-2
    Ns = 1e3
    # For the RUL prediction.
    max_ite = 60 # Maximun number of prediction states.
    max_RUL = 60 # RUL when not failure found.
    idx_start = 50
    step = 1
    idx_pred = np.arange(idx_ttf-idx_start, idx_ttf+step, step, dtype=int) # Index of the prediction instants.
    # Create the time.
    t_pred = np.arange(t[-1]+1, t[-1] + max_ite + 1, 1) 
    t_pred = np.concatenate((t, t_pred))
    
    xh, yh, y_bands, rul_mean, rul_bands, rul, rul_weights, pf = individual_battery_run(t, y, sigma_u, sigma_v, Ns, threshold, idx_ttf, idx_pred, t_pred, max_ite, max_RUL)

    # Save the result.
    file_name = 'result_' + name + '.pickle'
    with open(file_name, 'wb') as f:
        pickle.dump([t, y, threshold, idx_ttf, idx_pred, true_ttf, max_RUL, 
            xh, yh, y_bands, rul_mean, rul_bands, rul, rul_weights, t_pred,
            pf.particles, pf.w
        ], f, protocol=pickle.HIGHEST_PROTOCOL)

In [128]:
# Run particle filtering to estimate the state variables.
battery_list = ['CS2_35', 'CS2_36', 'CS2_37', 'CS2_38']
# Directly read from the archived data.
with open('data_all.pickle', 'rb') as f:
    data_all = pickle.load(f)

# CS2_35

In [129]:
from utility import drop_outlier_sw
import tensorflow
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd


In [130]:
df = data_all['CS2_38']
df1 = data_all['CS2_36']
df2 = data_all['CS2_37']


In [131]:
df1

In [132]:
!pip install seaborn

In [133]:
import seaborn as sns
sns.heatmap(df.corr(),annot=True)

In [134]:
df.fillna(method='ffill', inplace=True)
df1.fillna(method='ffill', inplace=True)
df2.fillna(method='ffill', inplace=True)

In [135]:
X = df['discharging capacity']
X1 = df1['discharging capacity']
X2 = df2['discharging capacity']


In [136]:
X

In [137]:
X1

In [138]:
idx = drop_outlier_sw(X,20)
idx1 = drop_outlier_sw(X1,20)
idx2 = drop_outlier_sw(X2,20)


x1 = X.iloc[idx]
x2 = X1.iloc[idx1]
x3 = X2.iloc[idx2]

In [139]:
x1

In [140]:
x2

X1.to_csv('X1.csv')
X2.to_csv('X2.csv')
X3.to_csv('X3.csv')

In [141]:
from sklearn.preprocessing import MinMaxScaler

ss = MinMaxScaler()

In [142]:
X1 = pd.DataFrame(x1)
X1 = ss.fit_transform(X1)
X1 = pd.DataFrame(X1)

X2 = pd.DataFrame(x2)
X2 = ss.fit_transform(X2)
X2 = pd.DataFrame(X2)

X3 = pd.DataFrame(x3)
X3 = ss.fit_transform(X3)
X3 = pd.DataFrame(X3)

X1 = X1[:832]
X2 = X2[:832]
X3 = X3[:832]

In [143]:
X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)

In [144]:
X1

In [145]:
X1=X1.flatten()
X2=X2.flatten()
X3=X3.flatten()

In [146]:
len(X3)

In [147]:
X1 = pd.DataFrame(X1[:832])
X2 = pd.DataFrame(X2[:864])
X3 = pd.DataFrame(X3[:928])



In [148]:
index_x1 = np.arange(0,len(X1))
index_x2 = np.arange(0,len(X2))
index_x3 = np.arange(0,len(X3))

In [149]:
index_x1 = pd.DataFrame(index_x1)
index_x2 = pd.DataFrame(index_x2)
index_x3 = pd.DataFrame(index_x3)

In [150]:
X_1_ = pd.concat([index_x1,X1],axis=1,ignore_index=True)
X_2_ = pd.concat([index_x2,X2],axis=1,ignore_index=True)
X_3_ = pd.concat([index_x3,X3],axis=1,ignore_index=True)

In [151]:
x_1 = pd.DataFrame(X_1_)
x_2 = pd.DataFrame(X_2_)
x_3 = pd.DataFrame(X_3_)

In [152]:
x_2

In [153]:
x_2[0]

In [154]:
yyy = np.arange(0,len(X1))
yyy_1 = np.arange(0,len(X2))
yyy_2 = np.arange(0,len(X3))

In [155]:
import numpy as np
import math
from scipy.optimize import curve_fit

def my_func(x, a, b, c, d):
    return a*np.exp(b*x)+c*np.exp(d*x)

popt, pcov = curve_fit(my_func, yyy,X1[0],p0= np.array([1.1, -5e-5, -1.5e-3, .006]),bounds = ((1, -1e-3, -2e-2, .001), (1.2, -2e-5, -1e-3, .01)))
popt1, pcov1 = curve_fit(my_func, yyy_1,X2[0], p0= np.array([1.1, -5e-5, -1.5e-3, .006]),bounds = ((1, -1e-3, -2e-2, .001), (1.2, -2e-5, -1e-3, .01)))
popt2, pcov2 = curve_fit(my_func, yyy_2,X3[0], p0= np.array([1.1, -5e-5, -1.5e-3, .006]),bounds = ((1, -1e-3, -2e-2, .001), (1.2, -2e-5, -1e-3, .01)))



##x_hat, _ = curve_fit(degradation_mdl,xdata=t_data, ydata=y_data, p0=x0, bounds=bounds)


##x0 = np.array([1.1, -5e-5, -1.5e-3, .006])    
##bounds = ((1, -1e-3, -2e-2, .001), (1.2, -2e-5, -1e-3, .01))



In [156]:
a,b,c,d= popt
popt

In [157]:
a1,b1,c1,d1 = popt1
popt1

In [158]:
a2,b2,c2,d2 = popt2
popt2

In [159]:
yyy_curve_fit = []

for x in yyy:
    yyy_curve_fit.append(a*np.exp(b*x)+c*np.exp(d*x))

yyy_curve_fit1 = []

for x in yyy_1:
    yyy_curve_fit1.append(a1*np.exp(b1*x)+c1*np.exp(d1*x))
    
yyy_curve_fit2 = []

for x in yyy_2:
    yyy_curve_fit2.append(a2*np.exp(b2*x)+c2*np.exp(d2*x))

In [160]:
plt.plot(yyy,X1[0])

plt.plot(yyy_1,X2[0])

plt.plot(yyy_2,yyy_curve_fit2)
plt.plot(yyy_2,X3[0])

In [161]:
plt.plot(yyy_1,yyy_curve_fit1)
plt.plot(yyy_1,X2[0])

In [162]:
plt.plot(yyy,X1[0])


In [163]:
plt.plot(yyy_2,yyy_curve_fit2)
plt.plot(yyy_2,X3[0])

In [164]:
ax = (a+a1+a2)/3
bx = (b+b1+b2)/3
cx = (c+c1+c2)/3

In [165]:
yyy_curve_fit3 = []
for x in yyy_2:
    yyy_curve_fit3.append(a*np.exp(b*x)+c*np.exp(d*x))

In [166]:
plt.plot(yyy,X1[0])

plt.plot(yyy_1,X2[0])

plt.plot(yyy_2,yyy_curve_fit2)

plt.plot(yyy_2,yyy_curve_fit3)

plt.plot(yyy_2,X3[0])

plt.plot()

In [167]:
def prepare_data(timeseries_data, n_features):
    X, y =[],[]
    for i in range(len(timeseries_data)):
		# find the end of this pattern
        end_ix = i + n_features
		# check if we are beyond the sequence
        if end_ix > len(timeseries_data)-1:
            break
		# gather input and output parts of the pattern
        seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_input = 32

In [168]:
X1, y1 = prepare_data(X1[0], n_input)
X2, y2 = prepare_data(X2[0], n_input)
X3, y3 = prepare_data(X3[0], n_input)

In [169]:
X2.shape

In [170]:
X1_ = X1.reshape((X1.shape[0], X1.shape[1], 1))
X2_ = X2.reshape((X2.shape[0], X2.shape[1], 1))
X3_ = X3.reshape((X3.shape[0], X3.shape[1], 1))

from keras.preprocessing.sequence import TimeseriesGenerator

df_rnn = df_rnn.reshape((len(df_rnn), 1))
generated_batches = TimeseriesGenerator(df_rnn, df_rnn, length=n_input, batch_size=1)

model_rnn = keras.Sequential()
model_rnn.add(keras.layers.LSTM(32, activation='selu', input_shape=(n_input, 1),return_sequences=True))
model_rnn.add(keras.layers.LSTM(32, activation='selu', input_shape=(n_input, 1),return_sequences=False))
model_rnn.add(keras.layers.Dense(64, activation='selu', kernel_regularizer=tf.keras.regularizers.l2(0.0002)))
model_rnn.add(keras.layers.Dense(32, activation='selu', kernel_regularizer=tf.keras.regularizers.l2(0.0002)))
model_rnn.add(keras.layers.Dense(1))

model_rnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate =1e-3), loss='mse')

In [171]:
k = 0

def expo(y_true,y_pred):
    global k 
    val = y_pred - k - (a * np.exp(b) + c )
    k = y_pred
    print(val + tf.math.abs(y_true-y_pred))
    print(tf.executing_eagerly())
    return val + tf.math.abs(y_true-y_pred)

    

        

In [172]:
def function(x):
    return (a*(x**2) + b*(x)+ c)

In [173]:
def huber_fn(y_true, y_pred):

    error = y_true - y_pred
    is_small_error = tensorflow.abs(error) < 1
    squared_loss = tensorflow.square(error) / 2 
    linear_loss  = tensorflow.abs(error) - 0.5
    print(y_pred.shape)
    print(tf.executing_eagerly())
    return tf.where(is_small_error, squared_loss, linear_loss)

a = (a+a1+a2)/3
b = (b+b1+b2)/3
c = (c+c1+c2)/3

In [174]:
x_1[1][2]

In [175]:
1 in x_1.values

In [176]:
from sklearn.metrics import mean_squared_error

cycle_arr = []
theoretical = []

def custom(y_true,y_pred):
    global cycle_arr
    global x_1
    for i in y_true:
        if i in x_1.values:
            cycle_arr.append(x_1[x_1[1]==i][0].iloc[0])
        if i in x_2.values:
            cycle_arr.append(x_2[x_2[1]==i][0].iloc[0])
        if i in x_3.values:
            cycle_arr.append(x_3[x_3[1]==i][0].iloc[0])
    
    if y_true[0] in x_1.values:
        for cycle in cycle_arr:
            theoretical.append([a*np.exp(b*(cycle))+c*np.exp(d*(cycle))])
        rmse = tensorflow.math.abs(y_true-y_pred)
        diff = tensorflow.math.abs((y_pred) - np.array(theoretical))
        error = diff + rmse*1.01
        print(tensorflow.executing_eagerly())
        cycle_arr.clear()
        theoretical.clear()
        print('x1')
        return error

    if y_true[0] in x_2.values:
        for cycle in cycle_arr:
            theoretical.append([a1*np.exp(b1*(cycle))+c1*np.exp(d1*(cycle))])
        rmse = tensorflow.math.abs(y_true-y_pred)
        diff = tensorflow.math.abs((y_pred) - np.array(theoretical))
        error = diff + rmse*1.01
        print(tensorflow.executing_eagerly())
        cycle_arr.clear()
        theoretical.clear()
        print('x2')
        return error

    if y_true[0] in x_3.values:
        for cycle in cycle_arr:
            theoretical.append([a2*np.exp(b2*(cycle))+c2*np.exp(d2*(cycle))])
        rmse = tensorflow.math.abs(y_true-y_pred)
        diff = tensorflow.math.abs((y_pred) - np.array(theoretical))
        error = diff + rmse*1.01
        print(tensorflow.executing_eagerly())
        cycle_arr.clear()
        theoretical.clear()
        print('x3')
        return error


In [177]:
x_2

cycle_arr1 = []
theoretical1 = []

def custom1(y_true,y_pred):
    global cycle_arr1
    global x_1
    for i in y_true:
        cycle_arr1.append(x_3[x_3[1]==i][0].iloc[0])
    for cycle in cycle_arr1:
        theoretical1.append([a1*(cycle)**2+b1*(cycle)+c1])
    rmse = tf.math.abs(y_true-y_pred)
    diff = tf.math.abs((y_pred) - np.array(theoretical1))
    error = diff + rmse
    print(tf.executing_eagerly())
    cycle_arr1.clear()
    theoretical1.clear()
    return error
    

cycle_arr2 = []
theoretical2 = []

def custom2(y_true,y_pred):
    global cycle_arr2
    global x_1
    for i in y_true:
        cycle_arr2.append(x_1[x_1[1]==i][0].iloc[0])
    for cycle in cycle_arr2:
        theoretical2.append([a2*(cycle)**2+b2*(cycle)+c2])
    rmse = tf.math.abs(y_true-y_pred)
    diff = tf.math.abs((y_pred) - np.array(theoretical2))
    error = diff + rmse
    print(tf.executing_eagerly())
    cycle_arr2.clear()
    theoretical2.clear()
    return error
    

In [178]:
from keras import Sequential
from keras.layers import Dense, Masking, LSTM


from numpy.random import seed
seed(42)



model_rnn = Sequential()
model_rnn.add(LSTM(32,
                return_sequences=False))

model_rnn.add(Dense(16, activation='relu'))

model_rnn.add(Dense(1, activation='linear'))

model_rnn.compile(optimizer='adam', loss=custom, metrics=['mse', 'mae', 'mape'],run_eagerly = True)

In [179]:
model_rnn.fit(X1_,y1, epochs=30, verbose=1)

In [180]:
model_rnn.fit(X2_,y2, epochs=30, verbose=1)

In [181]:
model_rnn.fit(X3_,y3, epochs=30, verbose=1)

model_rnn = keras.models.load_model('rul_pred_model')

In [182]:
test = data_all['CS2_35']
test = test.dropna()

In [183]:
test

In [184]:
test = test['discharging capacity']

In [185]:
test = pd.DataFrame(test)
test = ss.fit_transform(test)

In [186]:
last_train_batch = test[550:582]

last_train_batch = last_train_batch.reshape((1,n_input,1))

In [187]:
model_rnn.predict(last_train_batch)[0]

In [188]:
test[583]

In [189]:
len(test)

test_predictions = []

first_eval_batch = test[500:500+n_input]
current_batch = first_eval_batch.reshape((1, n_input, 1))

for i in range(len(test[500:800])):
    
    # get the prediction value for the first batch
    current_pred = model_rnn.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [190]:
test_predictions = []

test_predictions.extend(test[:n_input])

for i in range(len(test)-n_input+1):
    
    # get the prediction value for the first batch
    
    dummy = test[i:i+n_input].reshape((1, n_input, 1))
    # use the prediction to update the batch and remove the first value
    current_batch = (dummy)
    current_pred = model_rnn.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions.append(current_pred) 



In [191]:
test = ss.inverse_transform(test)
y_pred_future = ss.inverse_transform(test_predictions)

In [192]:
np.array(y_pred_future)

indexes_test_rnn = np.arange(500,800)
test_predictions = pd.DataFrame(test_predictions, index =indexes_test_rnn)
y_pred_future = pd.DataFrame(y_pred_future, index =indexes_test_rnn)


In [193]:
plt.figure(figsize = (12,8))

plt.plot(y_pred_future,linestyle='solid', marker='o', color='b', label='Predicted-inverse scaled')
plt.plot(test,linestyle='solid', marker='o', color='g', label='True')


plt.axhline(y = .7*1.1, color = 'r', linestyle = '--')




plt.title("Comparison - RNN", fontsize=18, fontweight="bold")
plt.legend()

plt.show()

In [194]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_pred_future[:-1], test, squared=False)

In [195]:
len(test)

In [196]:
from keras import Sequential
from keras.layers import Dense, Masking, LSTM


from numpy.random import seed
seed(42)



opt = keras.optimizers.Adam()
callback = keras.callbacks.EarlyStopping(monitor='loss',patience =3)


model = Sequential()
model.add(LSTM(32,return_sequences=False))
model.add(Dense(16, activation='relu'))

model.add(Dense(1, activation='linear'))

model.compile(optimizer=opt, loss='mse', metrics = ['mse', 'mae', 'mape', keras.metrics.RootMeanSquaredError(name='rmse')],run_eagerly = True)

In [197]:
model.fit(X1_,y1, epochs=30, verbose=1)

In [198]:
model.fit(X2_,y2, epochs=30, verbose=1)

In [199]:
model.fit(X3_,y3, epochs=30, verbose=1)

In [200]:
test = data_all['CS2_35']
test = test.dropna()

In [201]:
test = test['discharging capacity']
test = pd.DataFrame(test)
test = ss.fit_transform(test)

In [202]:
test_predictions_huber = []

test_predictions_huber.extend(test[:n_input])

for i in range(len(test)-n_input+1):
    
    # get the prediction value for the first batch
    
    dummy = test[i:i+n_input].reshape((1, n_input, 1))
    # use the prediction to update the batch and remove the first value
    current_batch = (dummy)
    current_pred = model.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions_huber.append(current_pred) 



In [203]:
test = ss.inverse_transform(test)
y_pred_future_huber = ss.inverse_transform(test_predictions_huber)

In [204]:
plt.figure(figsize = (12,8))

plt.plot(y_pred_future_huber,linestyle='solid', marker='o', color='b', label='Predicted-inverse scaled')
plt.plot(test,linestyle='solid', marker='o', color='g', label='True')






plt.title("Comparison - RNN", fontsize=18, fontweight="bold")
plt.legend()

plt.show()

In [205]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_pred_future_huber[:-1], test, squared=False)

In [206]:
mse_custom = []
for k in range(len(test)):
    mse_custom.append(mean_squared_error(y_pred_future[k],test[k]))

In [207]:
mse_huber = []
for k in range(len(test)):
    mse_huber.append(mean_squared_error(y_pred_future_huber[k],test[k]))

In [208]:
len(y_pred_future_huber)

In [209]:
len(y_pred_future)

In [210]:
len(test_predictions_huber)

In [211]:
len(test_predictions)

In [212]:
len(test)

In [213]:
plt.figure(figsize = (12,8))

plt.plot(mse_custom[100:150],linestyle='solid', marker='o', color='b', label='custom')
plt.plot(mse_huber[100:150],linestyle='solid', marker='o', color='g', label='huber')






plt.title("mse - error", fontsize=18, fontweight="bold")
plt.legend()

plt.show()

### RUL

In [214]:
from utility import drop_outlier_sw

name = battery_list[1]
battery = data_all[name]
battery.fillna(method='ffill', inplace=True)
# Get the time and degradation measurement. Perform filtering.
t1 = battery['cycle']
y1 = battery['discharging capacity']


rolling_window = 20
idx = drop_outlier_sw(y1, rolling_window)

t1 = np.array(t1[idx])
y1 = np.array(y1[idx])



name = battery_list[2]
battery = data_all[name]
battery.fillna(method='ffill', inplace=True)
# Get the time and degradation measurement. Perform filtering.
t2 = battery['cycle']
y2 = battery['discharging capacity']

rolling_window = 20
idx = drop_outlier_sw(y2, rolling_window)

t2 = np.array(t2[idx])
y2 = np.array(y2[idx])



name = battery_list[3]
battery = data_all[name]
battery.fillna(method='ffill', inplace=True)
# Get the time and degradation measurement. Perform filtering.
t3 = battery['cycle']
y3 = battery['discharging capacity']

rolling_window = 20
idx = drop_outlier_sw(y3, rolling_window)

t3 = np.array(t3[idx])
y3 = np.array(y3[idx])

In [215]:
true_ttf1, idx_ttf1 = cal_ttf(t1, y1, .7*1.1)
true_ttf2, idx_ttf2 = cal_ttf(t2, y2, .7*1.1)
true_ttf3, idx_ttf3 = cal_ttf(t3, y3, .7*1.1)


In [216]:
initial_rul1 = true_ttf1 - 32
initial_rul2 = true_ttf2 - 32
initial_rul3 = true_ttf3 - 32


In [217]:
### calculate RUL on sliding window

window_size = n_input = 32

X1 = []
Y1 = []

k=0
j=0

window_cycle = initial_rul1
while window_cycle > 0:
    window_cycle = initial_rul1-k
    window = np.array(y1[j:j+window_size])
    k+=1
    j+=1
    X1.append(window)
    Y1.append(window_cycle)
    
X1 = np.array(X1)
Y1 = np.array(Y1)


##############
X2 = []
Y2 = []

k=0
j=0

window_cycle = initial_rul2
while window_cycle > 0:
    window_cycle = initial_rul2-k
    window = np.array(y2[j:j+window_size])
    k+=1
    j+=1
    X2.append(window)
    Y2.append(window_cycle)
    
X2 = np.array(X2)
Y2 = np.array(Y2)

#############

X3 = []
Y3 = []

k=0
j=0

window_cycle = initial_rul3
while window_cycle > 0:
    window_cycle = initial_rul3-k
    window = np.array(y3[j:j+window_size])
    k+=1
    j+=1
    X3.append(window)
    Y3.append(window_cycle)
    
X3 = np.array(X3)
Y3 = np.array(Y3)



In [218]:
from sklearn.preprocessing import MinMaxScaler

ss1 = MinMaxScaler()
ss2 = MinMaxScaler()
ss3 = MinMaxScaler()

Y1 = pd.DataFrame(Y1)
Y1 = ss1.fit_transform(Y1)

Y2 = pd.DataFrame(Y2)
Y2 = ss1.fit_transform(Y2)

Y3 = pd.DataFrame(Y3)
Y3 = ss1.fit_transform(Y3)

In [219]:
X1 = X1[:608]
Y1 = Y1[:608]

X1 = X1.reshape((X1.shape[0], X1.shape[1], 1))
Y1 = Y1.reshape(Y1.shape[0],1)

X2 = X2[:608]
Y2 = Y2[:608]
X2 = X2.reshape((X2.shape[0], X2.shape[1], 1))
Y2 = Y2.reshape(Y2.shape[0],1)

X3 = X3[:768]
Y3 = Y3[:768]
X3 = X3.reshape((X3.shape[0], X3.shape[1], 1))
Y3 = Y3.reshape(Y3.shape[0],1)

    val = y_pred - m
    m = y_pred
    print(m)
    print(val+ tf.math.abs(y_true-y_pred))

In [220]:
len(X3)

In [221]:
m = 0

def loss(y_true,y_pred):
    global m 
    val = y_pred - m
    m = y_pred
    print((val))
    print(tensorflow.executing_eagerly())
    return (val) + tensorflow.math.abs((y_true-y_pred))

In [222]:
def huber_fn(y_true, y_pred):
    global m 
    val = y_pred - m
    m = y_true
    error = y_true - y_pred
    is_small_error = tensorflow.abs(error) < 1 + val
    squared_loss = tensorflow.square(error) / 2 + tensorflow.square(val)
    linear_loss  = tensorflow.abs(error) - 0.5
    print(tensorflow.executing_eagerly())
    return tensorflow.where(is_small_error, squared_loss, linear_loss)

In [223]:
def penalty(y_true,y_pred):
    a = 15
    b = 10
    err = y_true-y_pred
    is_negative_error = err < 0
    print(err)
    print(tensorflow.executing_eagerly())
    under = tensorflow.math.exp((-err/a))-1
    over = tensorflow.math.exp((err/b))-1
    print(over)
    print(tensorflow.executing_eagerly())
    return tf.where(is_negative_error, under, over)

In [224]:
from keras import Sequential
from keras.layers import Dense, Masking, LSTM, Dropout, BatchNormalization, Bidirectional


from numpy.random import seed
seed(42)



opt = tensorflow.keras.optimizers.Adam()
callback = tensorflow.keras.callbacks.EarlyStopping(monitor='loss',patience =3)


model_rul = Sequential()
model_rul.add((LSTM(16,return_sequences = False)))
model_rul.add(Dense(8, activation='relu'))
model_rul.add(Dense(1, activation='relu'))

model_rul.compile(optimizer='adam', loss='mae', metrics=['mse', 'mae', 'mape', tensorflow.keras.metrics.RootMeanSquaredError(name='rmse')],run_eagerly = True)

In [225]:
model_rul.fit(X1,Y1, epochs=10, verbose=2)

In [226]:
model_rul.fit(X2,Y2, epochs=10, verbose=2)

In [227]:
model_rul.fit(X3,Y3, epochs=10, verbose=2)

In [228]:
y_pred_future = np.array(y_pred_future)
test = np.array(test)

In [229]:
cycle = np.arange(1,len(y_pred_future))

In [230]:
true_test, idx_test = cal_ttf(cycle, y_pred_future,  .7*1.1)

In [231]:
true_test

In [232]:

rul_test = true_test - 32
rul_test

y_test = []
k=0

window_cycle = rul_test
while window_cycle > 0:
    window_cycle = rul_test-k
    y_test.append(window_cycle)
    k+=1
    
y_test = np.array(y_test)


In [233]:
rul_predictions = []

k=0
j=0

first_eval_batch = y_pred_future[:n_input]
current_batch = first_eval_batch.reshape((1, n_input, 1))

window_cycle = rul_test
while window_cycle >= 0:
    window_cycle = rul_test-k
    window = np.array(y_pred_future[j:j+32])
    current_batch = window.reshape((1, n_input, 1))
    current_pred = model_rul.predict(current_batch)[0]
    rul_predictions.append(current_pred) 
    j+=1
    k+=1
    

In [234]:
rul_pred = ss1.inverse_transform(rul_predictions)

In [235]:
rul_predictions

In [236]:
plt.figure(figsize = (12,8))

plt.plot(rul_pred,linestyle='--', marker='o', color='b', label='Predicted RUL')
plt.plot(y_test,linestyle='solid', marker='o', color='g', label='True RUL')


plt.title("Comparison - RNN Outlier removed", fontsize=18, fontweight="bold")
plt.legend()

plt.show()

In [237]:
y_pred_future_huber = np.array(y_pred_future_huber)
test = np.array(test)
cycle = np.arange(1,len(y_pred_future_huber))
true_test, idx_test = cal_ttf(cycle, y_pred_future_huber,  .7*1.1)

In [238]:
true_test

In [239]:
rul_predictions_custom = []

k=0
j=0

first_eval_batch = y_pred_future_huber[:n_input]
current_batch = first_eval_batch.reshape((1, n_input, 1))

window_cycle = rul_test
while window_cycle >= 0:
    window_cycle = rul_test-k
    window = np.array(y_pred_future_huber[j:j+32])
    current_batch = window.reshape((1, n_input, 1))
    current_pred = model_rul.predict(current_batch)[0]
    rul_predictions_custom.append(current_pred) 
    j+=1
    k+=1
    

In [240]:
rul_pred_custom = ss1.inverse_transform(rul_predictions_custom)

In [241]:
rul_predictions_custom

In [242]:
plt.figure(figsize = (12,8))

plt.plot(rul_pred_custom,linestyle='--', marker='o', color='b', label='Predicted RUL')
plt.plot(y_test,linestyle='solid', marker='o', color='g', label='True RUL')


plt.title("Comparison - RNN Outlier removed", fontsize=18, fontweight="bold")
plt.legend()

plt.show()

In [243]:
rul_predictions_test = []

k=0
j=0

first_eval_batch = test[:n_input]
current_batch = first_eval_batch.reshape((1, n_input, 1))

window_cycle = rul_test
while window_cycle >= 0:
    window_cycle = rul_test-k
    window = np.array(test[j:j+32])
    current_batch = window.reshape((1, n_input, 1))
    current_pred = model_rul.predict(current_batch)[0]
    rul_predictions_test.append(current_pred) 
    j+=1
    k+=1
    

In [244]:
rul_predictions_test = ss1.inverse_transform(rul_predictions_test)

In [245]:
plt.figure(figsize = (12,8))

plt.plot(rul_predictions_test,linestyle='--', marker='o', color='b', label='Predicted RUL')
plt.plot(rul_pred_custom,linestyle='--', marker='o', color='r', label='Predicted RUL custom')
plt.plot(y_test,linestyle='solid', marker='o', color='g', label='True RUL')
plt.plot(rul_pred,linestyle='--', marker='o', color='y', label='Predicted RUL smooth')


plt.title("Comparison - RNN Outlier removed", fontsize=18, fontweight="bold")
plt.legend()

plt.show()

In [246]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, rul_predictions_test[:-1], squared=False)


In [247]:
### NEURAL FILTERED

mean_squared_error(y_test, rul_pred[:-1], squared=False)

In [248]:
### NEURAL FILTERED custom loss



%load_ext jupyternotify



In [249]:
%%notify

mean_squared_error(y_test, rul_pred_custom[:-1], squared=False)

In [250]:
plt.figure(figsize = (12,8))

plt.plot(rul_pred,linestyle='--', marker='o', color='b', label='Predicted RUL')
plt.plot(y_test,linestyle='solid', marker='o', color='g', label='True RUL')

plt.xlabel('Cycles elapsed')
plt.ylabel('RUL')
plt.title("RUL Prediction for Traditional Method", fontsize=18, fontweight="bold")
plt.legend()

plt.show()

In [251]:
plt.figure(figsize = (12,8))

plt.plot(rul_pred_custom,linestyle='--', marker='o', color='r', label='Multi-Neural Method')
plt.plot(y_test,linestyle='solid', marker='o', color='g', label='True RUL')
plt.xlabel('Cycles elapsed')
plt.ylabel('RUL')
plt.title("RUL Prediction for Multi-Neural Method", fontsize=18, fontweight="bold")
plt.legend()


In [252]:
plt.figure(figsize = (12,8))

plt.plot(rul_predictions_test,linestyle='--', marker='o', color='b', label='Traditional Method')
plt.plot(rul_pred_custom,linestyle='--', marker='o', color='r', label='Multi-Neural Method')
plt.plot(y_test,linestyle='solid', marker='o', color='g', label='True RUL')
plt.title("RUL Comparison", fontsize=18, fontweight="bold")
plt.legend()
